In [3]:
import dalex as dx
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

In [4]:
data = dx.datasets.load_apartments()
print(data)


      m2_price  construction_year  surface  floor  no_rooms     district
1         5897               1953       25      3         1  Srodmiescie
2         1818               1992      143      9         5      Bielany
3         3643               1937       56      1         2        Praga
4         3517               1995       93      7         3       Ochota
5         3013               1992      144      6         5      Mokotow
...        ...                ...      ...    ...       ...          ...
996       6355               1921       44      2         2  Srodmiescie
997       3422               1921       48     10         2       Bemowo
998       3098               1980       85      3         3       Bemowo
999       4192               1942       36      7         1     Zoliborz
1000      3327               1992      112      6         5      Mokotow

[1000 rows x 6 columns]
